Our Sources of data are as follows: 


1st dataset: 
data.world - https://data.world/
https://data.world/noaa/severe-weather-data-hail-2018

2nd dataset:
https://data.world/noaa/severe-weather-data-hail-2017

3rd dataset 
#https://www.kaggle.com/tsnowak/us-geographic-codes
#US Geographic Codes Dataset

In [1]:
#import the requirements
import numpy as np
import pandas as pd
import datetime
import pyodbc
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()


In [2]:
# Import data sources 
counties = 'uszipfipscounty.csv'
hail2018 = "hail201802.csv"
hail2017 = "hail2017.csv"

In [3]:
# The correct encoding must be used to read the CSV in pandas
dfcounty = pd.read_csv(counties, encoding="ISO-8859-1")
dfhail2018 = pd.read_csv(hail2018, encoding="ISO-8859-1")
dfhail2017 = pd.read_csv(hail2017, encoding="ISO-8859-1")

Data Cleaning 
(cleaning, joining, filtering, aggregating, etc).

In [4]:
# Delete extraneous column
dfcounty.drop(['ansci','countycode'], axis=1, inplace=True)
dfcounty.head()

,zip,statename,state,countyname,fips,lat,long
0,501,New York,NY,Suffolk County,36103,40.943554,-72.692218
1,544,New York,NY,Suffolk County,36103,40.943554,-72.692218
2,6390,New York,NY,Suffolk County,36103,40.943554,-72.692218
3,11701,New York,NY,Suffolk County,36103,40.943554,-72.692218
4,11702,New York,NY,Suffolk County,36103,40.943554,-72.692218


In [5]:
#review data types
dfcounty.dtypes

zip             int64
statename      object
state          object
countyname     object
fips            int64
lat           float64
long          float64
dtype: object

In [6]:
dfhail2017['ztime']=pd.to_datetime(dfhail2017.ztime)

In [7]:
# Delete extraneous column
dfhail2017.drop(['azimuth','range','cellid','azimuth'], axis=1, inplace=True)
dfhail2017.head()

,ztime,long,lat,wsrid,sevprob,prob,maxsize
0,1970-01-01 05:36:10.400,-162.21326,22.81952,PHKI,-999,-999,-999.0
1,1970-01-01 05:36:10.400,-162.12851,22.79089,PHKI,-999,-999,-999.0
2,1970-01-01 05:36:10.400,-162.11157,22.78516,PHKI,-999,-999,-999.0
3,1970-01-01 05:36:10.400,-161.97694,22.78461,PHKI,-999,-999,-999.0
4,1970-01-01 05:36:10.400,-161.96012,22.77859,PHKI,-999,-999,-999.0


In [8]:
#adding breakout of date for further analysis
dfhail2017['year'] = pd.DatetimeIndex(dfhail2017['ztime']).year
dfhail2017['quarter'] = pd.DatetimeIndex(dfhail2017['ztime']).quarter
dfhail2017['month'] = pd.DatetimeIndex(dfhail2017['ztime']).month
dfhail2017['weekofyear'] = pd.DatetimeIndex(dfhail2017['ztime']).weekofyear
dfhail2017['time'] = pd.DatetimeIndex(dfhail2017['ztime']).time

dfhail2017.head()

,ztime,long,lat,wsrid,sevprob,prob,maxsize,year,quarter,month,weekofyear,time
0,1970-01-01 05:36:10.400,-162.21326,22.81952,PHKI,-999,-999,-999.0,1970,1,1,1,05:36:10.400000
1,1970-01-01 05:36:10.400,-162.12851,22.79089,PHKI,-999,-999,-999.0,1970,1,1,1,05:36:10.400000
2,1970-01-01 05:36:10.400,-162.11157,22.78516,PHKI,-999,-999,-999.0,1970,1,1,1,05:36:10.400000
3,1970-01-01 05:36:10.400,-161.97694,22.78461,PHKI,-999,-999,-999.0,1970,1,1,1,05:36:10.400000
4,1970-01-01 05:36:10.400,-161.96012,22.77859,PHKI,-999,-999,-999.0,1970,1,1,1,05:36:10.400000


In [9]:
dfhail2017.dtypes

ztime         datetime64[ns]
long                 float64
lat                  float64
wsrid                 object
sevprob                int64
prob                   int64
maxsize              float64
year                   int64
quarter                int64
month                  int64
weekofyear             int64
time                  object
dtype: object

In [10]:
# Delete extraneous column
dfhail2018.drop(['azimuth','range','cellid','azimuth'], axis=1, inplace=True)


In [18]:
dfhail2018['ztime']=pd.to_datetime(dfhail2018.ztime)

In [19]:
#adding breakout of date for further analysis
dfhail2018['year'] = pd.DatetimeIndex(dfhail2018['ztime']).year
dfhail2018['quarter'] = pd.DatetimeIndex(dfhail2018['ztime']).quarter
dfhail2018['month'] = pd.DatetimeIndex(dfhail2018['ztime']).month
dfhail2018['weekofyear'] = pd.DatetimeIndex(dfhail2018['ztime']).weekofyear
dfhail2018['time'] = pd.DatetimeIndex(dfhail2018['ztime']).time

dfhail2018.head()

,ztime,long,lat,wsrid,sevprob,prob,maxsize,year,quarter,month,weekofyear,time
0,1970-01-01 05:36:20.200,129.34744,25.74558,PODN,0,30,0.5,1970,1,1,1,05:36:20.200000
1,1970-01-01 05:36:20.200,128.70164,24.87586,PODN,40,80,0.5,1970,1,1,1,05:36:20.200000
2,1970-01-01 05:36:20.200,129.38126,25.73217,PODN,10,30,0.5,1970,1,1,1,05:36:20.200000
3,1970-01-01 05:36:20.200,129.50145,26.01843,PODN,0,10,0.5,1970,1,1,1,05:36:20.200000
4,1970-01-01 05:36:20.200,129.03393,25.84804,PODN,0,10,0.5,1970,1,1,1,05:36:20.200000


In [12]:
dfhail2018.dtypes

ztime         float64
long          float64
lat           float64
wsrid          object
sevprob         int64
prob            int64
maxsize       float64
year            int64
quarter         int64
month           int64
weekofyear      int64
time           object
dtype: object

In [13]:
new_df = pd.merge(dfcounty, dfhail2018, on=['long','lat'], how='outer')
new_df.head()

,zip,statename,state,countyname,fips,lat,long,ztime,wsrid,sevprob,prob,maxsize,year,quarter,month,weekofyear,time
0,501.0,New York,NY,Suffolk County,36103.0,40.943554,-72.692218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,544.0,New York,NY,Suffolk County,36103.0,40.943554,-72.692218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6390.0,New York,NY,Suffolk County,36103.0,40.943554,-72.692218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11701.0,New York,NY,Suffolk County,36103.0,40.943554,-72.692218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11702.0,New York,NY,Suffolk County,36103.0,40.943554,-72.692218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
new_df.count()

zip            41867
statename      41867
state          41867
countyname     41867
fips           41867
lat           123889
long          123889
ztime          82022
wsrid          82022
sevprob        82022
prob           82022
maxsize        82022
year           82022
quarter        82022
month          82022
weekofyear     82022
time           82022
dtype: int64

In [15]:
#new_df = pd.merge(dfcounty, dfhail2018, on=['long','lat'], how='outer')
#new_df.head()

In [16]:
#new_df.dropna(how='any')    #to drop if any value in the row has a nan

In [17]:
# Merge school and student dataframes
#complete_data = pd.merge(school, student, on=('school_name'))
#complete_data.head()